# Basic Convolution

Convolution is a means to compress information or data across a window of data. It can be in 1D or 2D. Here we demonstrate through simple code how the convolution works and how we would code it in vanilla element-wise operations.

In [5]:
import numpy as np
import torch
import torch.nn as nn

## Mathematics

For 1D convolution:

$$ y[n] = \sum_{k=0}^{M-1} h[k] \cdot x[n-k]$$

Where $M$ shows that the $h$ filter is bounded. 


For 2D convolution:

$$ Y[i,j] = \sum_{m=0}^{K-1}\sum_{n=0}^{K-1} H[m,n] \cdot X[i-m,j-n]$$

$X$ or $x$ are padded when they are out of bounds.

## Code

For 1D Convolution:

In [17]:
# Setup for simple 1D convolution

# Simple vector for 1D convolution
# Padded on 2 ends
vector_x = torch.tensor([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 0.0])

# Simple 1x3 kernel
kernel = torch.tensor([-1.0, 0.0, 1.0])

# Simple convolution function
def conv1d(x, k):
    # Get the length of the input and kernel
    x_len = len(x)
    k_len = len(k)

    # Calculate the length of the output
    out_len = x_len - k_len + 1

    # Initialize the output tensor
    out = torch.zeros(out_len)

    # Perform the convolution operation
    for i in range(out_len):
        for j in range(k_len):
            out[i] += x[i + j] * k[j]
    return out

# Perform the convolution
out = conv1d(vector_x, kernel)
print("Convolution output:", out)

Convolution output: tensor([ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2., -9.])


In [ ]:
# 1D Convolution using PyTorch
vector_x = torch.tensor([[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 0.0]])
vector_x = vector_x.unsqueeze(0) # Add batch and channel dimensions

# Reshape kernel to match PyTorch's expected input
kernel = torch.tensor([[[-1.0, 0.0, 1.0]]])  # (out_channels=1, in_channels=1, kernel_size=3)

conv1d = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, bias=False)
conv1d.weight.data = kernel
pytorch_1d = conv1d(vector_x)

print("PyTorch 1D output:", pytorch_1d)
print("PyTorch 1D size:", pytorch_1d.size())

# Size is batch x output_channels x length

PyTorch 1D output: tensor([[[ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2., -9.]]],
       grad_fn=<ConvolutionBackward0>)
PyTorch 1D size: torch.Size([1, 1, 10])


For 2D Convolution:

In [20]:
# Simple 2D matrix with zero-padding already applied
matrix_x = torch.tensor([
    [0.0,  0.0,  0.0,  0.0, 0.0],
    [0.0,  1.0,  2.0,  3.0, 0.0],
    [0.0,  4.0,  5.0,  6.0, 0.0],
    [0.0,  7.0,  8.0,  9.0, 0.0],
    [0.0,  0.0,  0.0,  0.0, 0.0]
])

# Simple 3x3 kernel (e.g., Sobel-like edge detector)
kernel_2d = torch.tensor([
    [-1.0, 0.0, 1.0],
    [-2.0, 0.0, 2.0],
    [-1.0, 0.0, 1.0]
])

# 2D convolution function (no batch or channels)
def conv2d(x, k):
    IX, IY = x.shape
    KX, KY = k.shape
    OX = IX - KX + 1
    OY = IY - KY + 1

    out = torch.zeros((OX, OY))

    for ox in range(OX):
        for oy in range(OY):
            for kx in range(KX):
                for ky in range(KY):
                    out[ox, oy] += x[ox + kx, oy + ky] * k[kx, ky]
    return out

# Perform 2D convolution
output_2d = conv2d(matrix_x, kernel_2d)

print("2D Convolution Output:\n", output_2d)

2D Convolution Output:
 tensor([[  9.,   6.,  -9.],
        [ 20.,   8., -20.],
        [ 21.,   6., -21.]])


In [21]:
# Input 2D matrix with padding already applied
matrix_x = torch.tensor([
    [0.0,  0.0,  0.0,  0.0, 0.0],
    [0.0,  1.0,  2.0,  3.0, 0.0],
    [0.0,  4.0,  5.0,  6.0, 0.0],
    [0.0,  7.0,  8.0,  9.0, 0.0],
    [0.0,  0.0,  0.0,  0.0, 0.0]
])

# Reshape input to match PyTorch's 2D conv format: (batch, channels, height, width)
matrix_x = matrix_x.unsqueeze(0).unsqueeze(0)  # shape: (1, 1, 5, 5)

# Define a 3x3 kernel, same as in the manual version
kernel_2d = torch.tensor([[[[-1.0, 0.0, 1.0],
                            [-2.0, 0.0, 2.0],
                            [-1.0, 0.0, 1.0]]]])  # shape: (out_channels=1, in_channels=1, 3, 3)

# Set up Conv2d layer
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)
conv2d.weight.data = kernel_2d  # manually set the weights

# Perform the convolution
pytorch_2d = conv2d(matrix_x)

print("PyTorch 2D output:\n", pytorch_2d)
print("PyTorch 2D size:", pytorch_2d.size())  # should be (1, 1, H_out, W_out)

PyTorch 2D output:
 tensor([[[[  9.,   6.,  -9.],
          [ 20.,   8., -20.],
          [ 21.,   6., -21.]]]], grad_fn=<ConvolutionBackward0>)
PyTorch 2D size: torch.Size([1, 1, 3, 3])
